# Omdena  - Milan Chapter Agrifoods
## AI for Sustainable agri-food systems: use of Satellite Imagery
### Data Exploration for industrial crop in Italy 2006-2021
#### Author: Maria Fisher 


The main objective of this study is to have gather information about crop production in Italy for the period of 2006-2021. 

Crop dataset used in this study was downloaded from the Italian National Institute of Statistics (Istat).



In [ ]:
import warnings 
warnings.filterwarnings("ignore")

import os
import pandas as pd
pd.options.display.float_format = "{:.2f}".format
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import seaborn as sns 
import scipy 
import sklearn
import geopandas as gpd
import pgeocode
import folium
import sys
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot, plot

In [ ]:
industrial_crop = pd.read_csv('industrial_crop_final_2006-2021.csv',skipinitialspace=True)
industrial_crop.head()

## Pre-processing dataset 

In [ ]:
def show_info(industrial_crop):
    print('DATASET SHAPE: ',industrial_crop.shape, '\n')
    print('-'*50)
    print('FEATURE DATA TYPES:')
    print(industrial_crop.info())
    print('\n', '-'*50)
    print('NUMBER OF UNIQUE VALUES PER FEATURE:', '\n')
    print(industrial_crop.nunique())
    print('\n', '-'*50)
    print('NULL VALUES PER FEATURE')
    print(industrial_crop.isnull().sum())
show_info(industrial_crop)

In [ ]:
industrial_crop.describe()

## Total industrial_crop production in 2006-2021

In [ ]:
industrial_crop = industrial_crop.rename(columns={"total_production":"production_tonnes"})

In [ ]:
plt.figure(figsize= (10,5))
sns.barplot(x= 'Year', y= 'production_tonnes',data =industrial_crop,palette='coolwarm')
plt.title('Total industrial crop production 2006-2021')
plt.xlabel('Year')
plt.ylabel('Total production (tonnes)')
plt.show()



## Cereal production by Cities 

In [ ]:
industrial_crop_region =industrial_crop.groupby(by =industrial_crop.City)['production_tonnes','City'].sum().reset_index().sort_values(by = 'production_tonnes', ascending = False).head(10)
industrial_crop_region

In [ ]:
plt.figure(figsize= (7,5))
sns.barplot(x=industrial_crop_region['production_tonnes'],y=industrial_crop_region['City'], orient='h', palette='coolwarm');
plt.title('Total industrial crop production 2006-2021 by City')
plt.xlabel('Total production')
plt.ylabel('Cities')
plt.show()

##industrial_crop highest production 2006-2021

In [ ]:
print(industrial_crop.Type_crop.max())
print(industrial_crop.Type_crop.value_counts())
print(industrial_crop.Type_crop.nunique())



In [ ]:
print(industrial_crop.total_ha.max())
print(industrial_crop.total_ha.min())
print(industrial_crop.total_ha.value_counts())
print(industrial_crop.total_ha.nunique())


In [ ]:
industrial_crop.drop(industrial_crop[industrial_crop['total_ha'] == 0.00 ].index , inplace=True)

In [ ]:
industrial_crop.describe().astype(int)

In [ ]:
# rename 

industrial_crop = industrial_crop.replace('valerian in greenhouses','valerian-ghouse')
industrial_crop = industrial_crop.replace('sweet basil in greenhouses','basil-ghouse')
industrial_crop = industrial_crop.replace('other aromatic, medicinal and culinary plants','aromatic-medicinal')
industrial_crop = industrial_crop.replace('parsley in greenhouses','parsley-ghouse')
industrial_crop = industrial_crop.replace('parsley in open field','parsley-field')
industrial_crop = industrial_crop.replace('other oilseed crops (excluding sesame)','oilseeds(No-sesame)')
industrial_crop = industrial_crop.replace('other textile crops','textile-crops')


In [ ]:
plt.figure(figsize=(7,3))
industrial_crop['Type_crop'].value_counts().plot.bar()
plt.title('Total findustrial crop production 2006-2021 by crop')
plt.ylabel('Total production (tonnes)')
plt.show()


## Subseting data

In [ ]:
industrial_crop

In [ ]:
industrial_crop_top10 = industrial_crop.apply(lambda row: row[industrial_crop['Type_crop'].isin(['valerian-ghouse', 'basil-ghouse', 'sunflower', 'rape', 
                                                                                                 'soya beans', 'hemp', 'parsley-field', 
                                                                                                  'parsley-ghouse', 'tobacco', 'flax'])])

industrial_crop_top10.head()

In [ ]:
Q1 = industrial_crop_top10.quantile(0.25)
Q3 = industrial_crop_top10.quantile(0.75)
IQR = Q3 - Q1
print(IQR)

In [ ]:
# Remove outliers
industrial_crop_top10 = industrial_crop_top10[~((industrial_crop_top10 < (Q1 - 1.5 * IQR)) | (industrial_crop_top10 > (Q3 + 1.5 * IQR))).any(axis=1)]
print(industrial_crop_top10.shape)

In [ ]:
#Box plot and histplot (crop production distribuition)

plt.figure(figsize=(13,5))

plt.subplot(1,2,1)
sns.boxplot(data=industrial_crop_top10, x='production_tonnes', y="Type_crop",  color = 'darkblue', dodge=False)

plt.title(f'Box Plot total industrial crop production ')

plt.subplot(1,2,2)
sns.histplot(data=industrial_crop_top10, x= 'production_tonnes',  color = 'darkblue', kde=True, bins = 80)
plt.title(f'Distribution total industrial crop production')

plt.show()



In [ ]:
#Box plot and histplot of total area(ha)

plt.figure(figsize=(16,5))

plt.subplot(1,2,1)
sns.boxplot(data=industrial_crop_top10, x='Year', y="total_ha",  color = 'darkblue', dodge=False)

plt.title(f'Box Plot total area (ha)')

plt.subplot(1,2,2)
sns.histplot(data=industrial_crop_top10, x= 'total_ha',  color = 'darkblue', kde=True, bins = 80)
plt.title(f'Distribution total total area (ha)')

plt.show()

In [ ]:
print(industrial_crop_top10.total_ha.max())
print(industrial_crop_top10.total_ha.min())
print(industrial_crop_top10.total_ha.value_counts())
print(industrial_crop_top10.total_ha.nunique())


In [ ]:
print(industrial_crop_top10.Fertilizers_tonnes.max())



In [ ]:
fig, ax = plt.subplots(3,1, sharex=True, figsize=(8,11))
fig.autofmt_xdate()

industrial_crop['Year'] = (industrial_crop['Year']).astype(str)

# Total crop production
crops =industrial_crop[['Year', 'production_tonnes']]
crops = crops.groupby('Year').mean()
sns.lineplot(data= crops, x="Year", y="production_tonnes", ax=ax[0])

# Total area
crops =industrial_crop[['Year', 'total_ha']]
crops = crops.groupby('Year').mean()
sns.lineplot(data= crops, x="Year", y="total_ha", ax=ax[1])

# Ferlizers
fertilizers =industrial_crop[['Year', 'Fertilizers_tonnes']]
fertilizers = fertilizers.groupby('Year').mean()

sns.lineplot(data= fertilizers, x="Year", y="Fertilizers_tonnes", ax=ax[2])




In [ ]:
#Crop harvested by year

plotcrop10 = sns.relplot(kind='line', data=industrial_crop_top10, x='Year', 
                         y='production_tonnes', hue='Type_crop', style='Type_crop', aspect=1.75)


In [ ]:
#Fertilizers distributed by year

plotfertilizers10 = sns.relplot(kind='line', data=industrial_crop_top10, x='Year', 
                         y='Fertilizers_tonnes', hue='Type_fertilizer', style='Type_fertilizer', aspect=1.75)

# Climate data analysis

|Param_Code|Param_Name|Resolution|Depth [m]|Units|
|:--------:|:--------:|:--------:|:-------:|:---:|
|GWETPROF|Profile Soil Moisture|0.5° x 0.625°|1.34 - 8.53|water-free [0], saturated soil[1]| 
|GWETTOP|Surface Soil Wetness|0.5° x 0.625°|0.00 - 0.05| water-free [0], saturated soil[1]|
|GWETROOT|Root Zone Soil Wetness|0.5° x 0.625°|0.10 - 1.00|water-free [0], saturated soil[1]|
|CLOUD_AMT|Cloud Amount|---|---|%|
|TS|Earth Skin Temperature|---|---|°C|
|PS|Surface Pressure|---|---|kPa|
|RH2M|Relative Humidity at 2 Meters|---|---|%|
|QV2M|Specific Humidity at 2 Meters|---|---|g/kg|
|PRECTOTCORR|Precipitation Corrected|---|---|mm/day|
|T2M_MAX|Temperature at 2 Meters Maximum|---|---|°C|
|T2M_MIN|Temperature at 2 Meters Minimum|---|---|°C|
|T2M_RANGE|Temperature at 2 Meters Range|---|---|°C|
|WS2M|Wind Speed at 2 Meters|---|---|m/s|


  

*Precipitation Corrected (PRECTOTCORR)- [The bias corrected average of total precipitation at the surface of the earth in water mass (includes water content in snow)]

*Source? https://power.larc.nasa.gov/#resources

In [ ]:

fig, ax = plt.subplots(3, 2, sharex=True, figsize=(20,10))

fig.suptitle('Climate in Italy 2006 - 2021')

industrial_crop['Year'] = (industrial_crop['Year']).astype(str)

# relative_humidity
sns.lineplot(data=industrial_crop, x="Year", y="RH2M", ax=ax[0,0])

# precipitation
sns.lineplot(data=industrial_crop, x="Year", y="PRECTOTCORR", ax=ax[0,1])

# temp_max
sns.lineplot(data=industrial_crop, x="Year", y="T2M_MIN", ax=ax[1,0])

# temp_min
sns.lineplot(data=industrial_crop, x="Year", y="T2M_MAX", ax=ax[1,1])

# surface_soil_wetness
sns.lineplot(data=industrial_crop, x="Year", y="GWETTOP", ax=ax[2,0])

# root_zone_wetness
sns.lineplot(data=industrial_crop, x="Year", y="GWETROOT", ax=ax[2,1])


In [ ]:
industrial_crop

In [ ]:
industrial_crop_model =industrial_crop_top10.drop(columns =['Year','PS','TS','GWETTOP', 'QV2M','WS2M','T2M_RANGE','GWETPROF','CLOUD_AMT','PRECTOTCORR','lat', 'lon'  ])
industrial_crop_model

In [ ]:
industrial_crop_model = industrial_crop_model.to_csv('industrial_crop_model.csv', index=False)

# References


http://dati.istat.it

https://maps.princeton.edu/catalog/stanford-mn871sp9778

https://www.crea.gov.it/documents/68457/0/ITACONTA+2020_ENG+DEF+xweb+%281%29.pdf/95c6b30a-1e18-8e94-d4ac-ce884aef76e8?t=1619527317576

https://seaborn.pydata.org/generated/seaborn.relplot.html

https://www.statisticshowto.com/variance-inflation-factor/

https://statisticsbyjim.com/regression/multicollinearity-in-regression-analysis/

https://lost-stats.github.io/Presentation/Figures/heatmap_colored_correlation_matrix.html

https://plotly.com/python/box-plots/

https://pycaret.org/